In [1]:
import pymysql
import pandas as df
import json
import re
import jieba

In [2]:
#连接数据库
conn = pymysql.connect('localhost','root','9432','companyb',charset='utf8')
cursor=conn.cursor()

In [ ]:
cursor.execute("select company,主要人员,股东信息 from companya")
a=cursor.fetchall()

In [ ]:
for i in range(len(a)):
    print('\r'+str(i),end='')
    rlist=[]
    ilist=[]
    if a[i][1]!='':        
        dlist=a[i][1].split(';')
        for d in dlist:
            if len(d.split('：'))<2:
                continue
            dicr={}
            dicr['post']=d.split('：')[0]
            dicr['name']=d.split('：')[1]
            rlist.append(dicr)
    if a[i][2]!='':
        dlist=a[i][2].split(';')
        for d in dlist:
            dici={}
            dici['type']=d.split(',')[0]
            dici['name']=d.split(',')[1]
            ilist.append(dici)
    
    try:
        cursor.execute("insert into com_team_investor values('%s','%s','%s');"%(a[i][0],json.dumps(rlist,ensure_ascii=False),json.dumps(ilist,ensure_ascii=False)))
    except Exception as e:
        print(e)
    if i%1000==0:
        try:
            conn.commit()
        except:
            print('失败')
conn.commit()
    

In [ ]:
"insert into com_team_investor values('%s','%s','%s');"%(a[i][0],json.dumps(rlist,ensure_ascii=False),json.dumps(ilist,ensure_ascii=False))

In [ ]:
for i in range(len(a)):
    print('\r'+str(i),end='')
    if a[i][1]=='':
        z=0
    elif re.search('\d+',a[i][1]):
        z=int(re.search('\d+',a[i][1]).group())
        if re.search('美|欧',a[i][1]):
            z*=6 
    else:
        z=0
    try:
        cursor.execute("insert into ca_zb values('%s',%d);"%(a[i][0],z))
    except Exception as e:
        print(e)
    if i%100==0:
        try:
            conn.commit()
        except:
            print('失败')
conn.commit()

In [ ]:
cursor.execute("select *,ca_zb.zb from com_team_investor_copy1 join ca_zb on com_team_investor_copy1.company=ca_zb.company")
a=cursor.fetchall()

In [ ]:
a[1]

In [ ]:
for i in range(len(a)):
    print('\r'+str(i),end='')
    if a[i][1]=='[]':
        continue
    else:
        for ry in json.loads(a[i][1]):
            try:
                cursor.execute("insert into ca_renyuan values ('%s','%s','%s','%d');"%(a[i][0],ry["name"],ry["post"],a[i][4]))
            except Exception as e:
                print(e)
    if i%1000==0:
        try:
            conn.commit()
        except:
            print("失败")
conn.commit()

In [ ]:
for i in range(len(a)):
    print('\r'+str(i),end='')
    if a[i][2]=='[]':
        continue
    else:
        for ry in json.loads(a[i][2]):
            try:
                cursor.execute("insert into ca_investor values ('%s','%s','%s','%d');"%(a[i][0],ry["name"],ry["type"],a[i][4]))
            except Exception as e:
                print(e)
    if i%1000==0:
        try:
            conn.commit()
        except:
            print("失败")
conn.commit()

In [ ]:
cursor.execute("select * from com_team_investor")
a=cursor.fetchall()

In [ ]:
a[0]

In [ ]:
for i in range(229936,len(a)):
    print('\r'+str(i),end='')
    if a[i][2]=='[]':
        continue
    else:
        for ry in json.loads(a[i][2]):
            try:
                if re.search(ry['name'],a[i][1]):
                    continue
            except Exception as e:
                print(e)
                continue
            try:
                cursor.execute("insert into com_investor_1 values ('%s','%s','%s');"%(a[i][0],ry["name"],ry["type"]))
            except Exception as e:
                print(e)
    if i%1000==0:
        try:
            conn.commit()
        except:
            print("失败")
conn.commit()

In [ ]:
cursor.execute("select companya.company,companya.主要人员,companya.股东信息,ca_zb.zb from companya join ca_zb on companya.company=ca_zb.company")
a=cursor.fetchall()

In [ ]:
a[2]

In [ ]:
for i in range(len(a)):
    print('\r'+str(i),end='')
    if a[i][1]=='':
        continue
    else:
        for ry in a[i][1].split(';'):
            try:
                cursor.execute("insert into ca_renyuan values ('%s','%s','%s','%d');"%(a[i][0],ry.split('：')[1],ry.split('：')[0],a[i][3]))
            except Exception as e:
                #print("insert into ca_renyuan values('%s','%s','%s','%d');"%(a[i][0],ry.split('：')[1],ry.split('：')[0],a[i][3]))
                print(e)
    if i%1000==0:
        try:
            conn.commit()
        except:
            print("失败")
conn.commit()

In [ ]:
a[2]

In [ ]:
for i in range(len(a)):
    print('\r'+str(i),end='')
    if a[i][2]=='':
        continue
    else:
        for ry in a[i][2].split(';'):
            try:
                cursor.execute("insert into ca_investor values ('%s', '%s', '%s', '%d');"%(a[i][0],ry.split(',')[1],ry.split(',')[0],a[i][3]))
            except Exception as e:
                print(e)
    if i%1000==0:
        try:
            conn.commit()
        except:
            print("失败")
conn.commit()

In [ ]:
dicj={}
for i in range(len(a)):
    dicj[a[i][0]]=a[i][-1]

In [ ]:
dicr={}
for i in range(len(b)):
    dicr[b[i][0]+str(b[i][4])]=b[i][-1]

In [ ]:
for i in range(3,4):
    d=json.loads(a[i][13])
    #print(d)
    n=a[i][-1]
    for j in d:
        if j["userInfo"]["name"]+str(n) in dicr.keys():
            j["userInfo"]["uid"]=dicr[j["userInfo"]["name"]+str(n)]
        else:
            j["userInfo"]["uid"]=0
    print("update mymodel_jg set team='"+json.dumps(d,ensure_ascii=False)+"' where id="+str(n).replace('\\n',' '))
    cursor.execute("update mymodel_jg set team='%s' where id=%d"%(json.dumps(d,ensure_ascii=False),n))
    conn.commit()
    print(i,n)

In [ ]:
dicr['张明1']

In [3]:
#com_investor_fzrr_done
cursor.execute("select * from com_inv")
a=cursor.fetchall()

In [10]:
jieba.lcut(a[20][0])

['TCL', '集团股份', '有限公司']

In [31]:
jieba.lcut('一汽实业总公司')

['一汽', '实业', '总公司']

In [ ]:
for i in range(20,len(a)):
    c1=jieba.lcut(a[i][0])
    c2=jieba.lcut(a[i][1])
    count=0
    for c in c1:
        if c in c2:
            count+=1
    if count/len(c1)<=0.6:
        
        try:
            cursor.execute("insert into com_inv_1 values ('%s', '%s', '%s', '%s', '%s');"%a[i])
        except Exception as e:
            print(e)
    else:
        print(a[i])
    if i//1000==0:
        conn.commit()
conn.commit()

In [ ]:
stoplist=['公司','有限公司','集团','集团股份','股份','总公司']
for i in range(len(a)):
    #print(i)
    ctag=1
    ctag1=1
    s1=a[i][0];s2=a[i][1]
    if re.search('（.+）',s2):
        ctag=0
        s2=re.sub('（.+）','',s2)
    if re.search('（.+）',s1):
        ctag1=0
        s1=re.sub('（.+）','',s1)
    c1=jieba.lcut(s1)
    c2=jieba.lcut(s2)
    count=0
    countn=0
    for c in c1[:]:
        if c not in stoplist:
            countn+=1
            if c in c2[ctag:]:
                count+=1
    #print(c1,c2,count,countn)
    if countn==0 or count/countn<=0.3:
        #print('ok',a[i])
        
        try:
            cursor.execute("insert into com_inv_3 values ('%s', '%s', '%s', '%s', '%s');"%a[i])
        except Exception as e:
            print(e)
        
    else:
        print('xx',a[i])
    if i%1000==0:
        conn.commit()
conn.commit()

In [35]:
conn.commit()

In [9]:
cursor.execute("select company,对外投资 from adding2 WHERE `对外投资` NOT LIKE '%[]%'")
a=cursor.fetchall()

In [20]:
len(a)

378676

In [12]:
for i in range(len(a)):
    ilist=json.loads(a[i][1])
    for inv in ilist:
        try:
            cursor.execute("insert into com_inv values ('%s', '%s', '%s', '%s', '%s');"%(a[i][0],inv["company"],inv["money"],inv["time"],inv["status"]))
        except Exception as e:
            print(e)
    if i/1000==0:
        print(i)
        conn.commit()
conn.commit()

0


In [19]:
cursor.execute("select * from com_b1 WHERE `变更记录` NOT LIKE '%[]%'")
a=cursor.fetchall()

In [29]:
cursor.execute("select com_inv_1.*,com_b1.注册资本,com_b1.成立日期,com_b1.变更记录 from com_inv_1 join com_b1 on com_inv_1.company=com_b1.company WHERE com_b1.`变更记录` NOT LIKE '%[]%'")
a=cursor.fetchall()

In [59]:
a[14][7]

'[{"time": "2017-11-06", "type": "注册资本", "after": "3000万元", "before": "500万元"}, {"time": "2017-11-06", "type": "投资人", "after": "1 一兰科技（北京）有限公司 法人股东2 兰助晴 自然人股东3 润兰科技（北京）有限公司 企业法人4 兰雨晴 自然人股东5 李龑 自然人股东", "before": "1 兰助晴 自然人股东2 润兰科技（北京）有限公司 企业法人3 京蒙青创（北京）信息技术有限公司 企业法人4 李龑 自然人股东"}, {"time": "2017-11-06", "type": "企业类型", "after": "其他有限责任公司", "before": "有限责任公司(自然人投资或控股)"}, {"time": "2017-09-25", "type": "投资人", "after": "1 兰助晴 自然人股东2 润兰科技（北京）有限公司 企业法人3 京蒙青创（北京）信息技术有限公司 企业法人4 李龑 自然人股东", "before": "1 兰助晴 自然人股东2 京蒙青创（北京）信息技术有限公司 企业法人3 李龑 自然人股东"}, {"time": "2017-08-18", "type": "经营范围", "after": "劳务派遣（劳务派遣经营许可证有效期至2019年10月26日）。技术开发、技术推广、技术转让、技术咨询、技术服务；教育咨询（中介服务除外）；会议服务；销售计算机、软件及辅助设备、机械设备。企业依法自主选择经营项目，开展经营活动；依法须经批准的项目，经相关部门批准后依批准的内容开展经营活动；不得从事本市产业政策禁止和限制类项目的经营活动。", "before": "劳务派遣（劳务派遣经营许可证有效期至2019年10月26日）。技术开发、技术推广、技术转让、技术咨询、技术服务、教育咨询（中介服务除外）；会议服务。企业依法自主选择经营项目，开展经营活动；依法须经批准的项目，经相关部门批准后依批准的内容开展经营活动；不得从事本市产业政策禁止和限制类项目的经营活动。"}, {"time": "2017-02-22", "type": "经营范围", "after": "劳务派遣（劳务派遣经营许可证有效期至2019年1

In [ ]:
clist=[]
for i in range(len(a)):
    #print(i)
    dic=json.loads(a[i][7])
    for d in dic:
        if d["type"]=="注册资本":
            print(i)
            if re.search('\d+',d["before"]):
                m1=float(re.search('\d+',d["before"]).group())
            else:
                m1=0
            if re.search('\d+',d["after"]):
                m2=float(re.search('\d+',d["after"]).group())
            else:
                m2=0
            time2=int(d["time"][:4])
            time1=int(a[i][3][:4])
            if time2>time1:
                ch=(m2-m1)/(time2-time1)
                clist.append(list(a[i][:-1])+[ch])
                print(list(a[i][:-1])+[ch])
                break

In [79]:
cj=json.dumps(clist,ensure_ascii=False)
with open('AnaInv.json','w',encoding='utf8') as f:
    f.write(cj)